## EXTRACCIÓN DE CARACTERÍSTICAS
Este notebook extrae glosarios para las distintas categorías de texto utilizando la librería gensim.

In [1]:
import pandas as pd
import gensim
from gensim import corpora
from gensim import models
from gensim.utils import simple_preprocess

Cargamos nuestro dataframe de noticias

In [2]:
noticias_dataframe = pd.read_csv('../Datos/noticias_train.csv')

Separamos nuestras noticias por categoría

In [3]:
deportes_dataframe = noticias_dataframe[noticias_dataframe["category"]=="sports"]
deportes_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30 entries, 0 to 29
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   index     30 non-null     int64 
 1   category  30 non-null     object
 2   n_doc     30 non-null     int64 
 3   title     30 non-null     object
 4   path      30 non-null     object
 5   link      30 non-null     object
 6   docs      30 non-null     object
 7   corpus    30 non-null     object
dtypes: int64(2), object(6)
memory usage: 2.1+ KB


In [4]:
salud_dataframe = noticias_dataframe[noticias_dataframe["category"]=="health"]
salud_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30 entries, 30 to 59
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   index     30 non-null     int64 
 1   category  30 non-null     object
 2   n_doc     30 non-null     int64 
 3   title     30 non-null     object
 4   path      30 non-null     object
 5   link      30 non-null     object
 6   docs      30 non-null     object
 7   corpus    30 non-null     object
dtypes: int64(2), object(6)
memory usage: 2.1+ KB


In [5]:
ciencia_dataframe = noticias_dataframe[noticias_dataframe["category"]=="science"]
deportes_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30 entries, 0 to 29
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   index     30 non-null     int64 
 1   category  30 non-null     object
 2   n_doc     30 non-null     int64 
 3   title     30 non-null     object
 4   path      30 non-null     object
 5   link      30 non-null     object
 6   docs      30 non-null     object
 7   corpus    30 non-null     object
dtypes: int64(2), object(6)
memory usage: 2.1+ KB


In [6]:
politica_dataframe = noticias_dataframe[noticias_dataframe["category"]=="politics"]
politica_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30 entries, 90 to 119
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   index     30 non-null     int64 
 1   category  30 non-null     object
 2   n_doc     30 non-null     int64 
 3   title     30 non-null     object
 4   path      30 non-null     object
 5   link      30 non-null     object
 6   docs      30 non-null     object
 7   corpus    30 non-null     object
dtypes: int64(2), object(6)
memory usage: 2.1+ KB


### Extracción de glosario mediante TFIDF


La forma más sencilla para obtener nuestro glosario es mediante el esquema *tf-idf*

In [7]:
def extrae_glosario_tf_idf(categoria_dataframe, size):
    doc_tokens = [simple_preprocess(corpus) for corpus in categoria_dataframe["corpus"]]
    dictionary = corpora.Dictionary(doc_tokens)
    bow_corpus = [dictionary.doc2bow(doc, allow_update=True) for doc in doc_tokens]
    tfidf = models.TfidfModel(bow_corpus)
    bow_tfidf = tfidf[bow_corpus]
    tfidf_dic = {dictionary.get(id): value for doc in bow_tfidf for id, value in doc}
    tfidf_list = [k for k, v in sorted(tfidf_dic.items(), key=lambda item: item[1], reverse = True)]
    return tfidf_list[:size]
    

In [13]:
glosarios = {'deportes': [],
             'salud': [],
             'ciencia': [],
             'salud': []}

In [14]:
glosarios['deportes'] = extrae_glosario_tf_idf(deportes_dataframe, 30)

In [15]:
glosarios['salud'] = extrae_glosario_tf_idf(salud_dataframe, 30)

In [16]:
glosarios['politica'] = extrae_glosario_tf_idf(politica_dataframe, 30)

In [17]:
glosarios['ciencia'] = extrae_glosario_tf_idf(ciencia_dataframe, 30)

In [ ]:
Guardamos los glosarios en .txt

In [21]:
inicial = "../Datos/glosario"
for g in glosarios.items():
    tipo = g[0]
    fname = f"{inicial}_{tipo}.txt"
    with open(fname, 'w') as f:
        for termino in g[1]:
            f.write(termino)
            f.write('\n')
        

In [24]:
glosario_tokens = simple_preprocess(''.join(str(glosarios['deportes'])))

In [25]:
print(glosario_tokens)

['mbappe', 'correr', 'exencion', 'seguidor', 'mans', 'resto', 'gaal', 'var', 'magnussen', 'steiner', 'gasto', 'cristiano', 'club', 'cambio', 'formato', 'gonzalez', 'gonzalo', 'marko', 'suarez', 'ktm', 'moto', 'boston', 'estabilidad', 'horford', 'lakers', 'pts', 'reb', 'wiggins', 'colombia', 'alpine']


In [ ]:
def similaridad_tf_idf(glosario, texto):
    tokens = simple_preprocess(''.join(str(glosarios['deportes'])))
    